### 一. BroadCast Join
1. `spark.sql.autoBroadcastJoinThreshold`参数  
spark有一个参数,`spark.sql.autoBroadcastJoinThreshold`默认为10M, 表示当被join的表小于等于10M时, 该表会被自动设置为广播变量, 并发送到所有的Executors节点上进行节点内的join. (A left join B,则B为被join的表, 只有B会被广播)  
2. 优缺点  
 优点: 这样做完全避免了map阶段的shuffle输出文件, reduce阶段的shuffle获取操作.避免了shuffle 开销   
 缺点: 由于广播变量, 需要Driver节点先collect整个Dataframe, 再从Driver节点发送这个Dataframe到每一个Executors, 这会造成Driver节点的网络压力极大

### 二. Shuffle Hash Join
1. 分区广播  
当执行语法"A join B"时, 若B的大小超过了"spark.sql.autoBroadcastJoinThreshold"的限制无法广播, 则尝试将表A按照join key的hash值重新分区, 然后再将B表的join key计算hash值重新分区, 再把B表每个的分区数据分别广播到相同的A表分区节点上进行join.  
这要求B标的平均分区大小不超过`spark.sql.autoBroadcastJoinThreshold`参数所配置的值
2. 优缺点  
相比全表广播, 广播分区的hash join避免了Driver段的网络压力  
但仍只是用于表B不是很大的情况

### 三. Shuffle Sort Join
1. 即用即丢的join  
当执行"A join B"的语法, 如果两张表都很大, 如果按照hash分区后发往同一个节点, 极有可能内存溢出. 因此, 可先将A和B表根据join key进行shuffle, 保证join keys相同的值被发往同一个分区, 然后再对分区内的数据进行排序, 排序后再对分区内的数据进行join  
<img src="img/sortshufflejoin.png" height="60%" width="60%">
因为分区后, 2个表的序列是有序的, 只要从头遍历, 碰到join key值相同的2个记录就输出. 可见shuffle后sort join, 每个join key完成连接就被丢弃, 不至于把某一侧表的数据全部加载进内存, 2个表的记录即用即丢, 不会产生OOM